In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
original_df = pd.read_csv(r'C:\Users\USER\Documents\Python\Nareshit data analysis\stats and ML\ML\26th- mlr\MLR\House_data.csv')

In [3]:
df = original_df.copy()
df.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639


In [4]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [5]:
df.shape

(21613, 21)

In [6]:
# Initialize two lists
unique_cols = []
repeated_cols = []

# Check each column
for col in df.columns:
    if df[col].is_unique:
        unique_cols.append(col)
    else:
        repeated_cols.append(col)

# Print the results
print("Columns with unique values only:", unique_cols)
print("Columns with repeated values:", repeated_cols)

Columns with unique values only: []
Columns with repeated values: ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [8]:
df.isnull().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [9]:
# Find out if any cell contains any discrepancy, ie, unwanted special characters.
import re

def find_special_chars(series):
    return series[series.astype(str).str.contains(r'[^a-zA-Z0-9\s.]', regex=True, na=False)]

# Check each column
special_chars_found = {col: find_special_chars(df[col]) for col in df.columns}

# Display results
for col, values in special_chars_found.items():
    if not values.empty:
        print(f"Column '{col}' contains special characters in the following rows:")
        print(values, "\n")

Column 'long' contains special characters in the following rows:
0       -122.257
1       -122.319
2       -122.233
3       -122.393
4       -122.045
          ...   
21608   -122.346
21609   -122.362
21610   -122.299
21611   -122.069
21612   -122.299
Name: long, Length: 21613, dtype: float64 



In [10]:
df_numeric = df.select_dtypes(include='number')
correlation_matrix = df_numeric.corr()

In [12]:
correlation_matrix['price'].to_frame().sort_values(by='price')

,price
zipcode,-0.053203
id,-0.016762
long,0.021626
condition,0.036362
yr_built,0.054012
sqft_lot15,0.082447
sqft_lot,0.089661
yr_renovated,0.126434
floors,0.256794
waterfront,0.266369


In [13]:
df = df.drop(['id','date'],axis=1)

In [14]:
x = df.drop(columns='price',axis=1)

In [15]:
y=df[['price']]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [17]:
lin_model = LinearRegression()
lin_model.fit(x_train, y_train)

LinearRegression()

In [18]:
cols = list(x_train.columns)
pmax = 1
while len(cols) > 0:
    X_1 = sm.add_constant(x_train[cols])
    model = sm.OLS(y_train, X_1).fit()
    p_values = model.pvalues.iloc[1:]  # exclude intercept
    pmax = p_values.max()
    feature_with_p_max = p_values.idxmax()
    if pmax > 0.05:
        cols.remove(feature_with_p_max)
    else:
        break

In [19]:
selected_features = cols
print("Selected Features:", selected_features)

Selected Features: ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


In [20]:
ols_model = sm.OLS(y_train, X_1[selected_features]).fit()
print(ols_model.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.904
Model:                            OLS   Adj. R-squared (uncentered):              0.904
Method:                 Least Squares   F-statistic:                          1.018e+04
Date:                Sat, 05 Apr 2025   Prob (F-statistic):                        0.00
Time:                        14:40:40   Log-Likelihood:                     -2.3590e+05
No. Observations:               17290   AIC:                                  4.718e+05
Df Residuals:                   17274   BIC:                                  4.720e+05
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [21]:
## Step 8: Build Final Model using selected features
final_model = LinearRegression()
final_model.fit(x_train[selected_features], y_train)

LinearRegression()

In [27]:
# Predict
y_pred = final_model.predict(x_test[selected_features])
actual = y_test.values.ravel()
predicted = y_pred.ravel()

test_pred = pd.DataFrame({
    'Actual price': actual,
    'Predicted price': predicted
})

test_pred['Error'] = test_pred['Actual price'] - test_pred['Predicted price']
test_pred['Absolute Error'] = test_pred['Error'].abs()
# Optional: Format to suppress scientific notation
pd.set_option('display.float_format', '{:,.2f}'.format)

print(test_pred.head())

   Actual price  Predicted price       Error  Absolute Error
0    297,000.00       368,769.93  -71,769.93       71,769.93
1  1,578,000.00     1,541,859.35   36,140.65       36,140.65
2    562,100.00       544,498.30   17,601.70       17,601.70
3    631,500.00       579,240.95   52,259.05       52,259.05
4    780,000.00       981,753.69 -201,753.69      201,753.69


In [26]:
# Evaluation
print("R2 Score:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Bias score:",final_model.score(x_train[selected_features], y_train))
print('Variance score:',final_model.score(x_test[selected_features], y_test))

R2 Score: 0.694932184042933
MSE: 36279967098.6187
RMSE: 190473.00884539704
Bias score: 0.7004769312123038
Variance score: 0.694932184042933
